# Montecarlo Simulations - Valuating financial derivatives

## Summary

* Montecarlo Valuation for Binomial
* Montecarlo Valuation for Diffusion processes

Monte Carlo simulation is one of the most important algorithms with several applications in many fields, including quantitative finance as one of the most important. Specifically in finance, it can be used to price derivatives from all kinds such as options, futures, swaps, variance swaps, amongs other exotic ones. 

It consists of the calculation of repeated random samplings to make numerical estimations of unknown parameters, which in the case that matters to us is the log return of a given asset, for continuos time return, and consequently its price at the end of a simulation. The final objective is to apply the law of large numbers to obtain a value that is close to the final price of the derivative after M simulations of several trading periods. 

In this task, computational power comes to hand as an important tool that allows us to perform thoursands of simulations of the M final payoffs of the derivative. In the following lines I'll process to demonstrate the application of this theory with a Python script.

## Import libraries

In [4]:
from math import exp, log, sqrt
import datetime as dt
import numpy as np
from opciones import Analysis
import matplotlib.pyplot as plt 

## Programming functions for general cases

In [7]:
def montecarlo_binominal(s0, strike, maturity, sigma, r, n, sims):
    """
    Calculates the price of an option call by passing a set of parameters. 
    - s0: Underlying price at t0.
    - strike: Contract value of the call option.
    - maturity: period of time until the expiration of the contract in annual terms.
    - sigma: estimated constant volatility in annual terms.
    - r: estimated constant interest rate in annual terms.
    - sims: number of M simulations of the payoff of the call option.
    """
    delta = maturity / n
    m = r - sigma**2 / 2
    u = exp(m * delta + sigma * sqrt(delta))
    d = exp(m * delta - sigma * sqrt(delta))
    R = exp(r*delta)
    p = (R-d) / (u-d)   

    sims_vector = []
    payoff_final = []
    for w in range(sims):
        ran = np.random.rand(n)
        sims_vector.append(ran)

        log_sims = []
        for sim in ran:
            if sim < p:
                ret = log(d)
                log_sims.append(ret)
            elif sim > p:
                ret = log(u)
                log_sims.append(ret)
        suma = s0 * exp(sum(log_sims))

        po = max((suma-strike),0) 
        payoff_final.append(po)

    call_mc = np.mean(payoff_final) * exp(-r*maturity)
    return call_mc